## price and shares framework

In [1]:
prices = cbind(c(5, 7, 6, 7), c(10, 11, 12, 8))

In [2]:
prices

5,10
7,11
6,12
7,8


In [3]:
dimnames(prices) = list(paste0("t",0:3), c("A", "B"))

In [4]:
prices

,A,B
t0,5,10
t1,7,11
t2,6,12
t3,7,8


In [5]:
# initial portfolio value (investment)
V_P0 = 1000

In [7]:
# num of assets
N = ncol(prices)
N

[1] 2

In [8]:
# equal weighting of assets
w = rep(1 / N, N)
w

[1] 0.5 0.5

In [9]:
# number of shares of each asset
lambda = w * V_P0 / prices["t0",]
lambda

A   B 
100  50

In [10]:
# empty asset values
V_assets <- matrix(0, nrow(prices), ncol(prices), dimnames=dimnames(prices))
V_assets

,A,B
t0,0,0
t1,0,0
t2,0,0
t3,0,0


In [11]:
for(i in 1:nrow(prices)){
    V_assets[i,] = prices[i,] * lambda
}
V_assets

,A,B
t0,500,500
t1,700,550
t2,600,600
t3,700,400


In [13]:
# portfolio value
V_P = rowSums(V_assets)
V_P

t0   t1   t2   t3 
1000 1250 1200 1100

In [15]:
# portfolio returns
R_t = diff(V_P) / V_P[1:3]
R_t

t1          t2          t3 
 0.25000000 -0.04000000 -0.08333333

In [16]:
weights = V_assets / V_P
weights

,A,B
t0,0.5000000,0.5000000
t1,0.5600000,0.4400000
t2,0.5000000,0.5000000
t3,0.6363636,0.3636364


## weights and returns framework

In [17]:
library(PerformanceAnalytics)

Loading required package: xts

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘PerformanceAnalytics’


The following objects are masked _by_ ‘.GlobalEnv’:

    prices, weights


The following object is masked from ‘package:graphics’:

    legend




In [19]:
data(edhec)
head(edhec)

           Convertible Arbitrage CTA Global Distressed Securities
1997-01-31                0.0119     0.0393                0.0178
1997-02-28                0.0123     0.0298                0.0122
1997-03-31                0.0078    -0.0021               -0.0012
1997-04-30                0.0086    -0.0170                0.0030
1997-05-31                0.0156    -0.0015                0.0233
1997-06-30                0.0212     0.0085                0.0217
           Emerging Markets Equity Market Neutral Event Driven
1997-01-31           0.0791                0.0189       0.0213
1997-02-28           0.0525                0.0101       0.0084
1997-03-31          -0.0120                0.0016      -0.0023
1997-04-30           0.0119                0.0119      -0.0005
1997-05-31           0.0315                0.0189       0.0346
1997-06-30           0.0581                0.0165       0.0258
           Fixed Income Arbitrage Global Macro Long/Short Equity
1997-01-31                 0.019

In [20]:
R = edhec["1997", 1:5]
colnames(R) = c("CA", "CTAG", "DS", "EM", "EMN")
R

               CA    CTAG      DS      EM    EMN
1997-01-31 0.0119  0.0393  0.0178  0.0791 0.0189
1997-02-28 0.0123  0.0298  0.0122  0.0525 0.0101
1997-03-31 0.0078 -0.0021 -0.0012 -0.0120 0.0016
1997-04-30 0.0086 -0.0170  0.0030  0.0119 0.0119
1997-05-31 0.0156 -0.0015  0.0233  0.0315 0.0189
1997-06-30 0.0212  0.0085  0.0217  0.0581 0.0165
1997-07-31 0.0193  0.0591  0.0234  0.0560 0.0247
1997-08-31 0.0134 -0.0473  0.0147 -0.0066 0.0017
1997-09-30 0.0122  0.0198  0.0350  0.0229 0.0202
1997-10-31 0.0100 -0.0098 -0.0064 -0.0572 0.0095
1997-11-30 0.0000  0.0133  0.0054 -0.0378 0.0041
1997-12-31 0.0068  0.0286  0.0073  0.0160 0.0066

In [21]:
# num of assets
N = ncol(R)
N

[1] 5

In [23]:
weights = xts(matrix(rep(1 / N, N), 1), as.Date("1996-12-31"))
colnames(weights) = colnames(R)
weights

            CA CTAG  DS  EM EMN
1996-12-31 0.2  0.2 0.2 0.2 0.2

In [24]:
V_0 = 1

In [26]:
bop_value = eop_value = matrix(0, 2, ncol(R))
bop_value

0,0,0,0,0
0,0,0,0,0


In [28]:
t = 1

In [29]:
bop_value[t,] = coredata(weights) * V_0
bop_value

0.2,0.2,0.2,0.2,0.2
0.0,0.0,0.0,0.0,0.0


In [30]:
eop_value[t,] = coredata(1 + R[t,]) * bop_value[t,]
eop_value

0.20238,0.20786,0.20356,0.21582,0.20378
0.00000,0.00000,0.00000,0.00000,0.00000


In [31]:
t = 2

In [32]:
bop_value[t,] = eop_value[t-1,]
bop_value

0.20000,0.20000,0.20000,0.20000,0.20000
0.20238,0.20786,0.20356,0.21582,0.20378


In [33]:
eop_value[t,] = coredata(1 + R[t,]) * bop_value[t,]
eop_value

0.2023800,0.2078600,0.2035600,0.2158200,0.2037800
0.2048693,0.2140542,0.2060434,0.2271506,0.2058382


In [34]:
bop_weights = eop_weights = matrix(0, 2, ncol(R))
bop_weights

0,0,0,0,0
0,0,0,0,0


In [35]:
for(t in 1:2){
    bop_weights[t,] = bop_value[t,] / sum(bop_value[t,])
    eop_weights[t,] = eop_value[t,] / sum(eop_value[t,])
}

In [36]:
bop_weights

0.200000,0.2000000,0.2000000,0.2000000,0.2000000
0.195839,0.2011419,0.1969808,0.2088446,0.1971937


In [37]:
eop_weights

0.1958390,0.2011419,0.1969808,0.2088446,0.1971937
0.1936464,0.2023282,0.1947562,0.2147071,0.1945622
